In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from scipy import signal
from scipy.io.wavfile import write as wavwrite
import datetime
import glob
import os
import random

関数

In [2]:
# フーリエ変換の初期設定
n_fft = 206 # データの取得幅
hop_length = int(n_fft*0.01) # 次の取得までの幅
fmin = 5
fmax = 50

In [3]:
# バンドパスフィルタの初期設定
fp = np.array([15, 40]) #通過域端周波数
fs = np.array([5, 50]) #阻止域端周波数
gpass = 3   #通過域端最大損失[dB]
gstop = 40   #阻止域最小損失[dB]

In [8]:
#CFARの初期設定
num_train = 800
num_guard = 400
rate_fa = 6e-3

In [14]:
np.log10(5)

0.6989700043360189

In [21]:
y, sr = librosa.audio.load(filelist[20], sr=200)

In [25]:
np.max(np.abs(y))

1.0

In [24]:
np.min(np.abs(y))

1.013279e-05

In [9]:
#alpha
num_train*(rate_fa**(-1/num_train) - 1)

5.13238911936984

In [5]:
def bandpass(x, sr, fp, fs, gpass, gstop):
    fn = sr / 2   #ナイキスト周波数
    wp = fp / fn  #通過域端周波数を正規化
    ws = fs / fn  #阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop) 
    b, a = signal.butter(N, Wn, "band")          
    y = signal.filtfilt(b, a, x)                  
    return y          

In [6]:
def detect_peaks(x, num_train, num_guard, rate_fa):   
    num_cells = x.size
    num_train_half = round(num_train / 2)
    num_guard_half = round(num_guard / 2)
    num_side = num_train_half + num_guard_half
 
    alpha = num_train*(rate_fa**(-1/num_train) - 1) # threshold factor
    
    
    peak_idx = []
    noise_idx = []
    
    for i in range(num_side, num_cells - num_side):
        #指定範囲内の最大値のインデックス番号を取得し、i-num_sideを足した数がiと同値なら省く。
        if i != i-num_side+np.argmax(x[i-num_side:i+num_side+1]): 
            continue
        
        sum1 = np.sum(x[i-num_side:i+num_side+1])
        sum2 = np.sum(x[i-num_guard_half:i+num_guard_half+1]) 
        p_noise = (sum1 - sum2) / num_train 
        threshold = alpha * p_noise
        
        if x[i] > threshold: 
            peak_idx.append(i)
    
    return peak_idx

ファイルの指定

In [15]:
#obs番号
obs = '1805'
#出力する日数
pick_num = 5

In [16]:
filelist = sorted(glob.glob(f'D:/whale/data/origindata/{obs}_2020/*.wav'))

In [9]:
filelist.pop(len(filelist)-1)
filelist.pop(0)

'D:/whale/data/origindata/1805_2020\\180725-000000.wav'

In [10]:
pick = random.sample(list([i for i in range(len(filelist))]), pick_num)

In [11]:
for i in pick:
    print(filelist[i])

D:/whale/data/origindata/1805_2020\190402-000000.wav
D:/whale/data/origindata/1805_2020\180820-000000.wav
D:/whale/data/origindata/1805_2020\190407-000000.wav
D:/whale/data/origindata/1805_2020\181224-000000.wav
D:/whale/data/origindata/1805_2020\190725-000000.wav


In [12]:
try:
    for number in pick:
        #wavファイルの読み込み
        y, sr = librosa.audio.load(filelist[number], sr=200)
        names = str(filelist[number])
        name = names[35:41]
        os.makedirs(f'D:/whale/data/{obs}/{name}')
        os.makedirs(f'D:/whale/data/{obs}/{name}/split', exist_ok=True)

        #バンドパスフィルタを適用
        data = bandpass(y, sr=sr, fp=fp, fs=fs, gpass=gpass, gstop=gstop)

        #CFAR
        peak_idx = detect_peaks(np.abs(data), num_train=num_train, num_guard=num_guard, rate_fa=rate_fa)

        #peak_idxを作成
        with open(f'D:/whale/data/{obs}/{name}/peak.txt', 'w',encoding='utf-8', newline='\n') as f:
            f.write(f'検出数：{str(len(peak_idx))} num_train={num_train} num_guard={num_guard}rate_fa={rate_fa}\n')
            for i in peak_idx:
                tm = datetime.timedelta(seconds=int(i/200))
                f.write(str(tm) + ' ' + str(i) +'\n')
                
        #pick_listの作成
        with open(f'D:/whale/data/{obs}/pick_list.txt', 'a', encoding='utf=8') as f:
            print(f'D:/whale/data/{obs}/{name}', file=f)

        #3sのwavファイルを出力
        for i in range(len(peak_idx)):
            nameint = ['0'] * 6
            nameint.append(str(i))
            for j in range(len(str(i))):
                del nameint[0]
            namea = ''.join(nameint)
            wavwrite(f'D:/whale/data/{obs}/{name}/split/{name}_{namea}.wav',sr, data[peak_idx[i]-300:peak_idx[i]+300])
except FileExistError as e:
    print(f'{obs}_{name}は実行済みです')